
## AI6128 Project 2 Task 3
## **Fast map matching** on Google Colab
### Chang Han Yong G2102556J

#### REFERENCE: 
Can Yang & Gyozo Gidofalvi (2018) Fast map matching, an algorithm integrating hidden Markov model with precomputation, International Journal of Geographical Information Science, 32:3, 547-570, DOI: 10.1080/13658816.2017.1400548

https://fmm-wiki.github.io/

In [ ]:
# https://fmm-wiki.github.io/docs/installation/ubuntu.html

# Install all the requirements with:
! sudo apt-get install libboost-dev libboost-serialization-dev \
gdal-bin libgdal-dev make cmake libbz2-dev libexpat1-dev swig python-dev

Reading package lists... Done
Building dependency tree       
Reading state information... Done
libboost-dev is already the newest version (1.65.1.0ubuntu1).
libboost-dev set to manually installed.
make is already the newest version (4.1-9.1ubuntu1).
make set to manually installed.
python-dev is already the newest version (2.7.15~rc1-1).
gdal-bin is already the newest version (2.2.3+dfsg-2).
libboost-serialization-dev is already the newest version (1.65.1.0ubuntu1).
libboost-serialization-dev set to manually installed.
libgdal-dev is already the newest version (2.2.3+dfsg-2).
cmake is already the newest version (3.10.2-1ubuntu2.18.04.2).
libbz2-dev is already the newest version (1.0.6-8.1ubuntu0.2).
libbz2-dev set to manually installed.
libexpat1-dev is already the newest version (2.2.5-3ubuntu0.7).
libexpat1-dev set to manually installed.
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'sudo apt autoremove' to remove it.
The foll

In [ ]:
! git clone https://github.com/cyang-kth/fmm.git

Cloning into 'fmm'...
remote: Enumerating objects: 5162, done.
remote: Counting objects: 100% (43/43), done.
remote: Compressing objects: 100% (33/33), done.
remote: Total 5162 (delta 14), reused 35 (delta 10), pack-reused 5119
Receiving objects: 100% (5162/5162), 15.33 MiB | 27.21 MiB/s, done.
Resolving deltas: 100% (3062/3062), done.


In [ ]:
# This installation cell will require approximately 6 minutes of build...

import os
# change working directory
os.chdir("fmm")

if not os.path.exists('build'):
  os.mkdir('build')
# ! mkdir build
os.chdir("build")
# ! cd build
! cmake ..
! make -j4
! sudo make install

-- CMAKE version 3.22.6
-- Set CMP0074 state to NEW
-- Set CMP0086 state to NEW
-- Set CMP0078 state to NEW
-- The C compiler identification is GNU 7.5.0
-- The CXX compiler identification is GNU 7.5.0
-- Detecting C compiler ABI info
-- Detecting C compiler ABI info - done
-- Check for working C compiler: /usr/bin/cc - skipped
-- Detecting C compile features
-- Detecting C compile features - done
-- Detecting CXX compiler ABI info
-- Detecting CXX compiler ABI info - done
-- Check for working CXX compiler: /usr/bin/c++ - skipped
-- Detecting CXX compile features
-- Detecting CXX compile features - done
-- 
No conda environment found in PATH!
PATH=/opt/bin:/usr/local/nvidia/bin:/usr/local/cuda/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/tools/node/bin:/tools/google-cloud-sdk/bin

-- Could NOT find Conda (missing: CONDA_PREFIX) 
-- Non conda exist, search library in default path
-- Found GDAL: /usr/lib/libgdal.so (found suitable version "2.2.3", minimum required is

In [ ]:
# Installation verification
! fmm

[info][fmm_app_config.cpp:49 ] Start reading FMM configuration from arguments
fmm argument lists:
--ubodt (required) <string>: Ubodt file name
--network (required) <string>: Network file name
--network_id (optional) <string>: Network id name (id)
--source (optional) <string>: Network source name (source)
--target (optional) <string>: Network target name (target)
--gps (required) <string>: GPS file name
--gps_id (optional) <string>: GPS id name (id)
--gps_x (optional) <string>: GPS x name (x)
--gps_y (optional) <string>: GPS y name (y)
--gps_timestamp (optional) <string>: GPS timestamp name (timestamp)
--gps_geom (optional) <string>: GPS geometry name (geom)
--gps_point (optional): if specified read input data as gps point, otherwise (default) read input data as trajectory
--output (required) <string>: Output file name
--output_fields (optional) <string>: Output fields
  opath,cpath,tpath,mgeom,pgeom,
  offset,error,spdist,tp,ep,length,duration,speed,all
-k/--candidates (optional) <int>

In [ ]:
# Run python script

# Change to the parent folder which contains fmm_test.py
# cd ../example/python
if os.getcwd() != "/content/fmm/example/python":
  os.chdir("/content/fmm/example/python")
os.system('python fmm_test.py')

256

# Main Section
# Fast Map Matching

In [ ]:
from fmm import Network,NetworkGraph,FastMapMatch,FastMapMatchConfig,UBODT

In [ ]:
network = Network("/content/fmm/example/data/edges.shp")
print("Nodes {} edges {}".format(network.get_node_count(),network.get_edge_count()))
graph = NetworkGraph(network)

Nodes 17 edges 30


In [ ]:
from fmm import UBODTGenAlgorithm

In [ ]:
ubodt_gen = UBODTGenAlgorithm(network,graph)

In [ ]:
status = ubodt_gen.generate_ubodt("/content/fmm/example/data/ubodt.txt", 4, binary=False, use_omp=True)
print(status)

Status: success
Time takes 0 seconds



In [ ]:
ubodt = UBODT.read_ubodt_csv("/content/fmm/example/data/ubodt.txt")

In [ ]:
model = FastMapMatch(network,graph,ubodt)

In [ ]:
k = 4
radius = 0.4
gps_error = 0.5
fmm_config = FastMapMatchConfig(k,radius,gps_error)

In [ ]:
wkt = "LINESTRING(0.200812146892656 2.14088983050848,1.44262005649717 2.14879943502825,3.06408898305084 2.16066384180791,3.06408898305084 2.7103813559322,3.70872175141242 2.97930790960452,4.11606638418078 2.62337570621469)"

In [ ]:
result = model.match_wkt(wkt,fmm_config)

In [ ]:
print("Matched path: ", list(result.cpath))
print("Matched edge for each point: ", list(result.opath))
print("Matched edge index ",list(result.indices))
print("Matched geometry: ",result.mgeom.export_wkt())
print("Matched point ", result.pgeom.export_wkt())

Matched path:  [8, 11, 13, 18, 20, 24]
Matched edge for each point:  [8, 11, 18, 18, 20, 24]
Matched edge index  [0, 1, 3, 3, 4, 5]
Matched geometry:  LINESTRING(0.20081215 2,1 2,2 2,3 2,3 3,4 3,4 2.6233757)
Matched point  LINESTRING(0.20081215 2,1.4426201 2,3 2.1606638,3 2.7103814,3.7087218 3,4 2.6233757)


In [ ]:
candidates = []
for c in result.candidates:
    candidates.append((c.edge_id,c.source,c.target,c.error,c.length,c.offset,c.spdist,c.ep,c.tp))

In [ ]:
import pandas as pd
df = pd.DataFrame(candidates,
                  columns=["eid","source","target","error","length","offset","spdist","ep","tp"])

In [ ]:
df.head()

,eid,source,target,error,length,offset,spdist,ep,tp
0,8,7,8,0.140890,1.0,0.200812,0.000000,0.961078,0.000000
1,11,8,5,0.148799,1.0,0.442620,1.241808,0.956684,1.000000
2,18,6,11,0.064089,1.0,0.160664,1.718044,0.991819,0.943813
3,18,6,11,0.064089,1.0,0.710381,0.549718,0.991819,1.000000
4,20,11,12,0.020692,1.0,0.708722,0.998340,0.999144,0.699640


In [ ]:
from fmm import GPSConfig,ResultConfig

In [ ]:
input_config = GPSConfig()
input_config.file = "../data/trips.csv"
input_config.id = "id"

In [ ]:
print(input_config.to_string())

gps file : ../data/trips.csv
id column : id
geom column : geom
timestamp column : timestamp
x column : x
y column : y
GPS point : false



In [ ]:
result_config = ResultConfig()
result_config.file = "../data/mr.txt"
result_config.output_config.write_opath = True
print(result_config.to_string())

Result file : ../data/mr.txt
Output fields: opath cpath mgeom 


In [ ]:
status = model.match_gps_file(input_config, result_config, fmm_config)
print(status)

Status: success
Time takes 0 seconds
Total points 17 matched 17
Total trajectories 3 matched 3
Map match percentage 1
Map match speed inf points/s 



# Main Section
# Fast Map Matching

In [ ]:
import os
import csv
from fmm import FastMapMatch,Network,NetworkGraph,UBODTGenAlgorithm,UBODT,FastMapMatchConfig
from fmm import Network,NetworkGraph,STMATCH,STMATCHConfig

In [ ]:
from google.colab import drive

drive.mount('/content/drive/') # mount your drive
data_dir = "/content/drive/MyDrive/Colab Notebooks/data/porto_by_city/"


Mounted at /content/drive/


In [ ]:
network = Network("/content/drive/MyDrive/Colab Notebooks/data/porto_by_city/edges.shp","fid", "u", "v")
print("Nodes {} edges {}".format(network.get_node_count(),network.get_edge_count()))
graph = NetworkGraph(network)

In [ ]:
if os.path.isfile("/content/drive/MyDrive/Colab Notebooks/data/porto_by_city/ubodt.txt"):
  ubodt = UBODT.read_ubodt_csv("/content/drive/MyDrive/Colab Notebooks/data/porto_by_city/ubodt.txt")
  print('Read from existing ubodt file')
else:
  print('Generate and read the ubodt file')
  ubodt_gen = UBODTGenAlgorithm(network,graph)
  status = ubodt_gen.generate_ubodt("/content/drive/MyDrive/Colab Notebooks/data/porto_by_city/ubodt.txt", 0.03, binary=False, use_omp=True)
  print(status)
  ubodt = UBODT.read_ubodt_csv("/content/drive/MyDrive/Colab Notebooks/data/porto_by_city/ubodt.txt")

Read from existing ubodt file


In [ ]:
fmm_model = FastMapMatch(network,graph,ubodt)
k = 16
radius = 0.005
gps_error = 0.0005
fmm_config = FastMapMatchConfig(k,radius,gps_error)

stmatch_model = STMATCH(network,graph)
k = 8
gps_error = 0.0005
radius = 0.005
vmax = 0.0003
factor = 1.5
stmatch_config = STMATCHConfig(k, radius, gps_error, vmax, factor)

train1000 = []
with open("/content/drive/MyDrive/Colab Notebooks/data/train-1000.csv","r") as csvfile:
    reader = csv.reader(csvfile)
    for line in reader:
        train1000.append(line[8])                         #POLYLINES EXTRACTION

results = []
for t_number in range(1,1001):
    gps_points = eval(train1000[t_number])
    wkt = 'LINESTRING('+','.join([' '.join([str(j) for j in i]) for i in gps_points])+')'
    result = fmm_model.match_wkt(wkt, fmm_config)
    if list(result.cpath)==[]:
        print(f'stmatching trajectory {t_number}')
        result = stmatch_model.match_wkt(wkt, stmatch_config)
    candidates = list(result.candidates)
    results.append(dict(
        cpath=str(list(result.cpath)), mgeom=result.mgeom.export_wkt(), opath=str(list(result.opath)),
        offset=str([c.offset for c in candidates]), length=str([c.length for c in candidates]),
        spdist=str([c.spdist for c in candidates])
        ))

# cpath, opath, offset, length, spdist, mgeom
with open("/content/drive/MyDrive/Colab Notebooks/data/fmm_result.csv","w") as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(["t_number", "cpath", "opath", "offset", "length", "spdist", "mgeom"])

    write_list = []
    for t_number in range(1000):
        tr = results[t_number]
        write_list.append([t_number+1, tr['cpath'], tr['opath'], tr['offset'], tr['length'], tr['spdist'], tr['mgeom']])
    writer.writerows(write_list)


stmatching trajectory 3
stmatching trajectory 16
stmatching trajectory 34
stmatching trajectory 58
stmatching trajectory 60
stmatching trajectory 87
stmatching trajectory 102
stmatching trajectory 128
stmatching trajectory 133
stmatching trajectory 225
stmatching trajectory 231
stmatching trajectory 266
stmatching trajectory 295
stmatching trajectory 303
stmatching trajectory 310
stmatching trajectory 326
stmatching trajectory 333
stmatching trajectory 362
stmatching trajectory 377
stmatching trajectory 389
stmatching trajectory 457
stmatching trajectory 461
stmatching trajectory 472
stmatching trajectory 521
stmatching trajectory 537
stmatching trajectory 558
stmatching trajectory 575
stmatching trajectory 608
stmatching trajectory 612
stmatching trajectory 634
stmatching trajectory 658
stmatching trajectory 676
stmatching trajectory 698
stmatching trajectory 699
stmatching trajectory 722
stmatching trajectory 739
stmatching trajectory 763
stmatching trajectory 806
stmatching trajecto